In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Trying to build an anime recommender based on 
    1) just a Tf-idf analysis of the synopsis for similar recommendations, 
    2) weighting numeric and categorical columns e.g. genre/studios/score for similar recommendations, 
    3) combine both recommenders together to get a weighted recommender, and 
    4) add in preferences in the form of the rating.

I'll be doing some exploratory data analysis along the way. This is my first notebook, so plz be kind :) 

First, importing in the necessary libraries and the list of animes. We use .head() to get a view of what we have. Some columns e.g. type are dropped as the type can be inferred from the number of episodes.

In [ ]:
anime = pd.read_csv('/kaggle/input/anime-list-for-recommendation-system-june-2021/anime_list.csv')

anime_cleaned = anime.copy().drop(columns = ['duration','airing','aired','background','premiered','licensors','producers','status','type'], axis = 1)
#as most TV episodes are about 25 min per ep, background is empty, and I don't care when it's aired/premiered. 
anime_cleaned.head()


Let's try to use the TFIDF vectorizer to vectorise the synopsis and find those with the closest relation.

In [ ]:
tfidfvec = TfidfVectorizer(stop_words = 'english')
anime_cleaned['synopsis']=anime_cleaned['synopsis'].fillna('')
tfidf_matrix = tfidfvec.fit_transform(anime_cleaned['synopsis'])
tfidf_matrix.shape


Using the linear kernel to compute similarity:

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim=linear_kernel(tfidf_matrix,tfidf_matrix)
cosine_sim.shape

Now it's time to create a reverse mapping of titles to the index that they have.

In [ ]:
indices = pd.Series(anime_cleaned.index,index = anime_cleaned['title']).drop_duplicates()
print(indices[:5])

Defining a recommender function to 

In [ ]:
def recommender(title, cosine_sim = cosine_sim):
    #get the index based on title
    index = indices[title];
    #get cosine similarity for title
    cosine_sim_for_title = list(enumerate(cosine_sim[index]))
    #sort by cosine similarity
    sorted_by_second = sorted(cosine_sim_for_title, key = lambda x:x[1])
    chosen = sorted_by_second[1:6]
    anime_indices = [i[0] for i in chosen]
    return anime_cleaned['title'].iloc[anime_indices]

Let's see what they recommend for Attack on Titan.

In [ ]:
recommender('Shingeki no Kyojin')
#Attack on Titan's Japanese name

Part 1 down, part 2, 3 and 4 to go. The recommender function can be improved by looking at other aspects, not just the synopsis. Things like the genre, rating, etc. 

Need to do some exploratory analysis first on this dataset.

In [ ]:
import matplotlib.style as style
style.use('fivethirtyeight')
sns.histplot(anime_cleaned.score, bins =50)

As can be seen, the score for many animes is -1 as it's a null value. So I'll plot the data distribution of ratings of anime ignoring null values to obtain a more representative distribution of scores across animes.

When doing the eventual recommender, we will need to impute the values of null values. Ideally they will be randomly distributed, assuming a normal distribution of mean mu and standard deviation sigma respectively. However, dealing with different columns will need different null values - score will be the random distribution, popularity, scored_by will both be replaced with 0.

In [ ]:
toreplace = ['score','episodes','rank','popularity','scored_by']
for torepl in toreplace:
    anime_cleaned[torepl].replace(to_replace= -1,value = np.nan,inplace = True)
    
sns.histplot(anime_cleaned['score'], bins = 50)
print(anime_cleaned.score.isna().sum())

In [ ]:
mu = anime_cleaned['score'].mean()
sigma=np.std(anime_cleaned.score)

print(mu)
print(anime_cleaned['score'].quantile(0.80))
print(sigma)